In [ ]:
import json
from pathlib import Path
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Wczytywanie konfiguracji
config_path = Path("../config.json")
with open(config_path, "r") as config_file:
    config = json.load(config_file)

# Ścieżki z konfiguracji
model_file = Path(config["resnet18_masterCategory_model"])
test_file = Path(config["test_file"])
dataset_path = Path(config["dataset_path"])

print(f"Ścieżki załadowane:\nModel: {model_file}\nTest File: {test_file}\nDataset Path: {dataset_path}")


In [ ]:
# Wczytanie testowych danych
df_test = pd.read_csv(test_file)

# Tworzenie pełnych ścieżek do obrazów
df_test["image_path"] = df_test["id"].apply(lambda x: dataset_path / "images" / f"{x}.jpg")

# Przygotowanie etykiet
categories = df_test["masterCategory"].unique()
category_to_index = {category: idx for idx, category in enumerate(categories)}
index_to_category = {idx: category for category, idx in category_to_index.items()}
df_test["category_index"] = df_test["masterCategory"].map(category_to_index)

print("Pierwsze wiersze danych testowych:")
print(df_test.head())


In [ ]:
# Ustawienia dla generatora danych
image_size = (224, 224)  # Rozmiar obrazu zgodny z modelem
batch_size = 32

# Funkcja pomocnicza do wczytywania i przetwarzania obrazów
def preprocess_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, image_size)
    image = image / 255.0  # Normalizacja
    return image, label

# Konwersja do TensorFlow Dataset
test_ds = tf.data.Dataset.from_tensor_slices(
    (df_test["image_path"].astype(str), df_test["category_index"])
)
test_ds = test_ds.map(preprocess_image).batch(batch_size).prefetch(tf.data.AUTOTUNE)


In [ ]:
# Wczytanie modelu
resnet18_masterCategory_model = tf.keras.models.load_model(model_file)
resnet18_masterCategory_model.summary()


In [ ]:
# Ewaluacja modelu na zbiorze testowym
loss, accuracy = resnet18_masterCategory_model.evaluate(test_ds, verbose=1)
print(f"Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")


In [ ]:
# Predykcja na danych testowych
predictions = []
true_labels = []
for images, labels in test_ds:
    preds = resnet18_masterCategory_model.predict(images)
    predictions.extend(np.argmax(preds, axis=1))
    true_labels.extend(labels.numpy())

# Konwersja indeksów na kategorie
predicted_categories = [index_to_category[idx] for idx in predictions]
true_categories = [index_to_category[idx] for idx in true_labels]

# Dodanie predykcji do DataFrame
df_test["predicted_category"] = predicted_categories

# Pierwsze wiersze z wynikami
print(df_test[["masterCategory", "predicted_category"]].head())


In [ ]:
# Macierz konfuzji
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(true_categories, predicted_categories, labels=categories)

# Wyświetlenie macierzy
plt.figure(figsize=(10, 8))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=categories)
disp.plot(cmap=plt.cm.Blues, xticks_rotation="vertical")
plt.title("Macierz konfuzji")
plt.show()


In [ ]:
# Błędne klasyfikacje
df_test["is_correct"] = df_test["masterCategory"] == df_test["predicted_category"]
incorrect = df_test[~df_test["is_correct"]]

# Wyświetlenie kilku przykładów błędnych klasyfikacji
print("Przykłady błędnych klasyfikacji:")
print(incorrect[["image_path", "masterCategory", "predicted_category"]].head())

# Wizualizacja kilku obrazów błędnie sklasyfikowanych
plt.figure(figsize=(15, 10))
for i, row in enumerate(incorrect.sample(6).itertuples()):
    img = tf.image.decode_jpeg(tf.io.read_file(row.image_path))
    img = tf.image.resize(img, image_size)
    
    plt.subplot(2, 3, i + 1)
    plt.imshow(img / 255.0)
    plt.title(f"True: {row.masterCategory}\nPred: {row.predicted_category}")
    plt.axis("off")
plt.tight_layout()
plt.show()


In [ ]:
# Obliczenie metryk
from sklearn.metrics import classification_report

report = classification_report(true_categories, predicted_categories, target_names=categories)
print("Raport klasyfikacji:")
print(report)
